In [1]:
import pandas as pd
from glob import iglob
from itertools import chain
import tensorflow as tf
import numpy as np
from os import path
import os
from sklearn.preprocessing import OneHotEncoder

tf.__version__

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.6.0'

In [2]:
data_dir = path.join(os.getenv('AZ_BATCHAI_INPUT_DATASET'), 'imagenet')

In [3]:
train_df = pd.DataFrame.from_csv(path.join(data_dir, 'train.csv'))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [4]:
train_df = train_df.assign(filenames=train_df.filenames.apply(lambda x: path.join(data_dir,'train', x)))

In [5]:
train_df.head()

,filenames,wnid,num_id
0,/mnt/input/imagenet/train/n02119789/n02119789_...,n02119789,1
1,/mnt/input/imagenet/train/n02119789/n02119789_...,n02119789,1
2,/mnt/input/imagenet/train/n02119789/n02119789_...,n02119789,1
3,/mnt/input/imagenet/train/n02119789/n02119789_...,n02119789,1
4,/mnt/input/imagenet/train/n02119789/n02119789_...,n02119789,1


In [6]:
validation_df = pd.DataFrame.from_csv(path.join(data_dir, 'validation.csv'))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [7]:
validation_df = validation_df.assign(filenames=validation_df.filenames.apply(lambda x: path.join(data_dir,'validation', x)))

In [8]:
validation_df.head()

,filenames,num_id
0,/mnt/input/imagenet/validation/ILSVRC2012_val_...,490
1,/mnt/input/imagenet/validation/ILSVRC2012_val_...,361
2,/mnt/input/imagenet/validation/ILSVRC2012_val_...,171
3,/mnt/input/imagenet/validation/ILSVRC2012_val_...,822
4,/mnt/input/imagenet/validation/ILSVRC2012_val_...,297


In [9]:
oh_encoder = OneHotEncoder(sparse=False)

In [10]:
train_df[['num_id']].values.shape

(1281167, 1)

In [11]:
train_labels = oh_encoder.fit_transform(train_df[['num_id']].values).astype(np.uint8)

In [12]:
validation_labels = oh_encoder.transform(validation_df[['num_id']].values).astype(np.uint8)

In [13]:
validation_labels.shape

(50000, 1000)

In [14]:
tf.logging.set_verbosity(tf.logging.INFO)

In [15]:
train_data = tf.data.Dataset.from_tensor_slices((train_df['filenames'].values, train_labels))

In [16]:
WIDTH = 224
HEIGHT = 224
CHANNELS = 3
LR = 0.0001
EPOCHS = 5
BATCHSIZE = 32

In [17]:
IMAGENET_RGB_MEAN_CAFFE = np.array([123.68, 116.78, 103.94], dtype=np.float32)
IMAGENET_SCALE_FACTOR_CAFFE = 0.017

In [18]:
imagenet_mean = IMAGENET_RGB_MEAN_CAFFE
imagenet_scaling = IMAGENET_SCALE_FACTOR_CAFFE
width = WIDTH
height = HEIGHT
buffer=10
batch_size=BATCHSIZE
data_length = len(train_df)
validation_length = len(validation_df)

In [19]:
def _preprocess_image_labels(filename, label):
        # load and preprocess the image
        img_decoded = tf.to_float(tf.image.decode_png(tf.read_file(filename), channels=3))
        img_decoded = tf.image.resize_images(img_decoded, [WIDTH, HEIGHT])
        img_centered = tf.subtract(img_decoded, imagenet_mean)
        img_rgb = img_centered * imagenet_scaling
        return img_rgb, tf.cast(label, dtype=tf.float32)

In [20]:
def _parse_function_train(filename, label):
        img_rgb, label = _preprocess_image_labels(filename, label)
        # Random crop (from 264x264)
        img_rgb = tf.random_crop(img_rgb, [height, width, 3])
        # Random flip
        img_rgb = tf.image.random_flip_left_right(img_rgb)
        # Channels-first
        img_rgb = tf.transpose(img_rgb, [2, 0, 1])
        return img_rgb, label

In [21]:
def _parse_function_train(filename, label):
        img_rgb, label = _preprocess_image_labels(filename, label)
        # Channels-first
        img_rgb = tf.transpose(img_rgb, [2, 0, 1])
        return img_rgb, label

In [22]:
train_data_transform = tf.contrib.data.map_and_batch(_parse_function_train, batch_size)

In [23]:
train_data = (train_data.shuffle(len(train_df))
                        .repeat()
                        .apply(train_data_transform)
                        .prefetch(buffer))

In [24]:
validation_data = tf.data.Dataset.from_tensor_slices((validation_df['filenames'].values, validation_labels))

In [25]:
validation_data_transform = tf.contrib.data.map_and_batch(_parse_function_train, batch_size)

In [26]:
validation_data = (validation_data.repeat()
                        .apply(validation_data_transform)
                        .prefetch(buffer))

In [27]:
def train_input_fn():
    return train_data.make_one_shot_iterator().get_next()

In [28]:
def validation_input_fn():
    return validation_data.make_one_shot_iterator().get_next()


In [29]:
def model_fn(features, labels, mode, params):
    # Args:
    #
    # features: This is the x-arg from the input_fn.
    # labels:   This is the y-arg from the input_fn,
    #           see e.g. train_input_fn for these two.
    # mode:     Either TRAIN, EVAL, or PREDICT
    # params:   User-defined hyper-parameters, e.g. learning-rate.
    
    # Reference to the tensor named "x" in the input-function.
    x = features

    # The convolutional layers expect 4-rank tensors
    # but x is a 2-rank tensor, so reshape it.
    net = tf.reshape(x, [-1, width, height, CHANNELS])    

    # First convolutional layer.
    net = tf.layers.conv2d(inputs=net, name='b1_conv1',
                           filters=64, kernel_size=3,
                           padding='same', activation=tf.nn.relu)
    net = tf.layers.conv2d(inputs=net, name='b1_conv2',
                           filters=64, kernel_size=3,
                           padding='same', activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)

    # Second convolutional layer.
    net = tf.layers.conv2d(inputs=net, name='b2_conv1',
                           filters=128, kernel_size=3,
                           padding='same', activation=tf.nn.relu)
    net = tf.layers.conv2d(inputs=net, name='b2_conv2',
                           filters=128, kernel_size=3,
                           padding='same', activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)  
    
    # Third convolutional layer.
    net = tf.layers.conv2d(inputs=net, name='b3_conv1',
                           filters=256, kernel_size=3,
                           padding='same', activation=tf.nn.relu)
    net = tf.layers.conv2d(inputs=net, name='b3_conv2',
                           filters=256, kernel_size=3,
                           padding='same', activation=tf.nn.relu)
    net = tf.layers.conv2d(inputs=net, name='b3_conv3',
                           filters=256, kernel_size=3,
                           padding='same', activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2) 

    # Flatten to a 2-rank tensor.
    net = tf.layers.flatten(net)
    
    net = tf.layers.dense(inputs=net, name='layer_fc1',
                          units=2048, activation=tf.nn.relu)    

    net = tf.layers.dense(inputs=net, name='layer_fc2',
                          units=1000)

    # Logits output of the neural network.
    logits = net

    # Softmax output of the neural network.
    y_pred = tf.nn.softmax(logits=logits)
    
    # Classification output of the neural network.
    y_pred_cls = tf.argmax(y_pred, axis=1)

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': y_pred_cls,
            'probabilities': y_pred,
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=predictions)
    
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels,
                                                                       logits=logits)
    loss = tf.reduce_mean(cross_entropy)
        
    
    if mode == tf.estimator.ModeKeys.EVAL:
        accuracy = tf.metrics.accuracy(labels=tf.argmax(labels, axis=1),
                                   predictions=y_pred_cls,
                                   name='acc_op')
        metrics = {'accuracy': accuracy}
        tf.summary.scalar('accuracy', accuracy[1])
        return tf.estimator.EstimatorSpec(
                mode=mode,
                eval_metric_ops=metrics,
                loss=loss)

    # Define the optimizer for improving the neural network.
    optimizer = tf.train.AdamOptimizer(learning_rate=params["learning_rate"])

    # Get the TensorFlow op for doing a single optimization step.
    train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())


    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=loss,
        train_op=train_op)

In [30]:
params = {"learning_rate": 1e-4}

In [31]:
# !rm -r /mnt/model/*

In [32]:
rfig = tf.estimator.RunConfig(save_checkpoints_steps=1000)

In [33]:
model = tf.estimator.Estimator(model_fn=model_fn,
                               params=params,
                               model_dir=os.getenv('AZ_BATCHAI_OUTPUT_MODEL'),
                               config=rfig)

INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_tf_random_seed': None, '_num_ps_replicas': 0, '_task_type': 'worker', '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7faba5e44550>, '_num_worker_replicas': 1, '_task_id': 0, '_evaluation_master': '', '_service': None, '_model_dir': '/mnt/model', '_save_checkpoints_secs': None, '_save_checkpoints_steps': 1000, '_session_config': None, '_global_id_in_cluster': 0, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_master': ''}


In [34]:
for epoch in range(EPOCHS):
    model.train(input_fn=train_input_fn, steps=10) #data_length//batch_size
    print('evaluating')
    model.evaluate(input_fn=validation_input_fn, steps=10) #validation_length//batch_size

INFO:tensorflow:Calling model_fn.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /mnt/model/model.ckpt.
INFO:tensorflow:loss = 6.9115324, step = 1
INFO:tensorflow:Saving checkpoints for 10 into /mnt/model/model.ckpt.
INFO:tensorflow:Loss for final step: 6.907812.
evaluating
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-09-13:19:39
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /mnt/model/model.ckpt-10
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow

ResourceExhaustedError: /mnt/model/model.ckpt-41_temp_923ce82845fc492eb0423b507375e179/part-00000-of-00001.data-00000-of-00001.tempstate18363108291152576982; No space left on device
	 [[Node: save/SaveV2 = SaveV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](save/ShardedFilename, save/SaveV2/tensor_names, save/SaveV2/shape_and_slices, b1_conv1/bias/_371, b1_conv1/bias/Adam/_373, b1_conv1/bias/Adam_1/_375, b1_conv1/kernel/_377, b1_conv1/kernel/Adam/_379, b1_conv1/kernel/Adam_1/_381, b1_conv2/bias/_383, b1_conv2/bias/Adam/_385, b1_conv2/bias/Adam_1/_387, b1_conv2/kernel/_389, b1_conv2/kernel/Adam/_391, b1_conv2/kernel/Adam_1/_393, b2_conv1/bias/_395, b2_conv1/bias/Adam/_397, b2_conv1/bias/Adam_1/_399, b2_conv1/kernel/_401, b2_conv1/kernel/Adam/_403, b2_conv1/kernel/Adam_1/_405, b2_conv2/bias/_407, b2_conv2/bias/Adam/_409, b2_conv2/bias/Adam_1/_411, b2_conv2/kernel/_